# Part 2 - Ray-Tracing Intermediate Report
Use this notebook to draft the narrative for your next intermediate report. Replace the prompts in each section with your own text.

## 1. Input Wave Conditions
The values below reuse the EVA outputs from `Part_1_Mean_EVA_Waves/index_project_EVA_1.ipynb` (ResourceCode hindcast at the Bretagne Sud 1 site, 47.3236 deg N / -3.5522 deg E). Heights and directions come straight from the BM and POT fits so the ray-tracing report references the same forcing.
### 1-year storm setup
| Parameter | Block Maxima (BM) | Peak-over-Threshold (POT) | Notes |
| --- | --- | --- | --- |
| Source / dataset | EVA fit to 27 annual maxima in the notebook | EVA fit to 137 declustered peaks above threshold | Both rely on the ResourceCode hindcast node collocated with Bretagne Sud 1. |
| Offshore location | Bretagne Sud 1 (47.3236 deg N, -3.5522 deg E) | Same | Deep-water point west of the ray-tracing domain edge. |
| Significant wave height (H<sub>s</sub>) | 6.96 m (95% CI 6.42-7.96 m) for the 1.01-year return | 8.51 m (95% CI 8.22-8.76 m) for the 1.01-year return | Table reproduced from Part 1 (BM - POT = -1.55 m at this horizon). |
| Wave period (T<sub>p</sub>) | 13.9 s median of BM extremes (10%-90%: 12.8-18.3 s); T ~= 12.7sqrt(H/g) cross-check -> 10.7 s | 13.9 s median of POT extremes (10%-90%: 12.2-18.0 s); T ~= 12.7sqrt(H/g) cross-check -> 11.8 s | Periods fall within the mid-teens expected for energetic Atlantic swell. |
| Wave direction | Mean 255.9 deg (coming from WSW), circular std 11.6 deg | Mean 260.4 deg (WSW), circular std 12.6 deg | Both directions point toward the coastline, matching buoy climatology. |
| Reasonableness check | Moderate-depth swell propagating shoreward; BM amplitude slightly lower because it samples one event per year. | Larger H<sub>s</sub> reflects the fatter POT tail but is still within recorded extremes. | Either forcing option launches deep-water rays that refract toward shore with physical H-T pairs. |

### 50-year storm setup
| Parameter | Block Maxima (BM) | Peak-over-Threshold (POT) | Notes |
| --- | --- | --- | --- |
| Source / dataset | Same EVA workflow focused on the BM tail | Same EVA workflow focused on the POT tail | Extreme values cross over so both are documented for the ray model. |
| Offshore location | Bretagne Sud 1 (47.3236 deg N, -3.5522 deg E) | Same | Identical launch section for both storm severities. |
| Significant wave height (H<sub>s</sub>) | 12.84 m (95% CI 11.45-13.64 m) at 50-year return | 11.82 m (95% CI 10.51-13.08 m) at 50-year return | BM exceeds POT by +1.02 m at this horizon; both lie within each other's CI. |
| Wave period (T<sub>p</sub>) | Use the upper tail of BM quantiles (15-18 s) for propagation; T ~= 12.7sqrt(H/g) gives 14.5 s as a consistency check. | POT quantiles suggest 14-18 s, and the T ~= 12.7sqrt(H/g) check with H = 11.82 m gives 13.9 s. | Longer periods keep the waves in deep water at the ray-launch boundary. |
| Wave direction | Retain BM mean 255.9 deg (WSW) for the severe case | Retain POT mean 260.4 deg (WSW) | Severe storms share the same propagation sector; only H<sub>s</sub> and T differ. |
| Reasonableness check | H<sub>s</sub> > 12 m with T ~15 s is energetic but in line with hindcast extremes and remains within deep/intermediate water at the launch section. | POT offers a slightly lower H<sub>s</sub>, giving a conservative alternative forcing. | Both satisfy the expected storm climatology (Atlantic swell impinging from the west-southwest). |

### Additional considerations
- Retain the notebook CSV exports (e.g., `summary_bm_pot.csv`) with the full return-level table in case reviewers need to trace the numbers.
- Document any subsequent tweaks (e.g., if you refine T using dispersion in the ray code) so the intermediate report states which combination feeds the simulations.


## 2. Ray-Tracing Runs and Quality Checks
Describe the simulations you executed for each case.
- **Coverage:** _Do the traced rays span the region of interest? Do they originate in deep water?_
- **Input alignment:** _Do ray headings match your intended incident directions?_
- **Validation tests (if needed):** _If the default runs showed no/limited refraction, document any alternative parameters explored to confirm the model can capture refraction._

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from ocean_wave_tracing import Wave_tracing
import cmocean

### Load Bathymetry

In [ ]:
# bathymetry file path
base_path = Path.cwd() 
fname = base_path / 'GMRTv4_4_0_20251107topo.asc' # bathymetry file

# load bathymetry
with fname.open() as fid:
    ncols = int(fid.readline().split()[1])
    nrows = int(fid.readline().split()[1])
    lonll = float(fid.readline().split()[1])
    latll = float(fid.readline().split()[1])
    dd = float(fid.readline().split()[1])
dx = dd*6378.e3*np.pi/180*np.cos(np.pi/180*latll)
dy = dd*6378.e3*np.pi/180
bathy = -np.flipud(np.loadtxt(fname,skiprows=6))

# paths for created figs

fig_dir = base_path / "figures"
fig_dir.mkdir(parents=True, exist_ok=True)

print(f"Bathymetry loaded. Figures will be saved to: {fig_dir}")

### 1 Year Return Period

In [ ]:
# define parameters for 1 year return period
period_one_year= 13.9 # median wave period (s) for 1 year return period
mean_wave_direction = ((259.9 + 260.4)/2) - 180 # wave direction (deg)
ang_one_year   = mean_wave_direction
nr_case1    = 100  # number of wave rays for Case 1(less rays so its easier to see the main direction of the rays)
nr_case2    = 350  # number of wave rays for Case 2(more rays so the energy flux resolution is better)
simtime = 3600*2.5 # propagate 2.5 hour
nt    = 15000 # number of time steps, try to increase to limit edge cases

In [ ]:
# ray tracing for Case 1 (less rays so its easier to see the main direction of the rays)
ang = 90-ang_one_year # convert
wt_case1 = Wave_tracing(U=np.zeros((nrows,ncols)),
                  V=np.zeros((nrows,ncols)),
                  nx=ncols, ny=nrows, nt=nt, T=simtime,
                  dx=dx,dy=dy,nb_wave_rays=nr_case1,
                  domain_X0=0, domain_XN=dx*(ncols-1),
                  domain_Y0=0, domain_YN=dy*(nrows-1),
                  d=bathy)
if np.mod(ang,360)<45 or np.mod(ang,360)>=315:
    iws = 'left'
elif np.mod(ang,360)<135 and np.mod(ang,360)>=45:
    iws = 'bottom'
elif np.mod(ang,360)<225 and np.mod(ang,360)>=135:
    iws = 'right'
else:
    iws = 'top'
wt_case1.set_initial_condition(wave_period=period_one_year,
                         theta0=+np.pi/180*ang,
                         incoming_wave_side=iws)
wt_case1.solve()

In [ ]:
wt_case2 = Wave_tracing(U=np.zeros((nrows,ncols)),
                  V=np.zeros((nrows,ncols)),
                  nx=ncols, ny=nrows, nt=nt, T=simtime,
                  dx=dx,dy=dy,nb_wave_rays=nr_case2,
                  domain_X0=0, domain_XN=dx*(ncols-1),
                  domain_Y0=0, domain_YN=dy*(nrows-1),
                  d=bathy)
if np.mod(ang,360)<45 or np.mod(ang,360)>=315:
    iws = 'left'
elif np.mod(ang,360)<135 and np.mod(ang,360)>=45:
    iws = 'bottom'
elif np.mod(ang,360)<225 and np.mod(ang,360)>=135:
    iws = 'right'
else:
    iws = 'top'
wt_case2.set_initial_condition(wave_period=period_one_year,
                         theta0=+np.pi/180*ang,
                         incoming_wave_side=iws)
wt_case2.solve()

In [ ]:
# plotting wave rays --> Case 1
fig, ax = plt.subplots()
ax.set_aspect('equal')
cmap = cmocean.cm.deep.reversed()
cmap.set_bad(color='gray')
pc=ax.pcolormesh(wt_case1.x,wt_case1.y,-wt_case1.d,shading='auto',cmap=cmap)
fig.colorbar(pc)
for ray_id in range(wt_case1.nb_wave_rays):
    ax.plot(wt_case1.ray_x[ray_id,:],wt_case1.ray_y[ray_id,:],'-k')
ax.set_xlim((0,dx*(ncols-1)))
ax.set_ylim((0,dy*(nrows-1)))
pc.set_clim((-np.max(wt_case1.d),0))
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title('Wave rays')
fig.savefig(fig_dir / "wave_rays_1_year.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# wave energy flux density estimation --> Case 2
from scipy.interpolate import RegularGridInterpolator
xx,yy,hm=wt_case1.ray_density(x_increment=20,y_increment=20)
interp = RegularGridInterpolator(points=(xx[0,:],yy[:,0]),values=hm.transpose(),bounds_error=False,fill_value=None)
xg,yg=np.meshgrid(wt_case1.x,wt_case1.y,indexing='ij')
e = interp((xg,yg))
e = e.transpose()
e[np.isnan(wt_case1.d)]=np.nan
if np.mod(ang,360)<45 or np.mod(ang,360)>=315:
    escale = e[round(nrows/2),0]
elif np.mod(ang,360)<135 and np.mod(ang,360)>=45:
    escale = e[0,round(ncols/2)]
elif np.mod(ang,360)<225 and np.mod(ang,360)>=135:
    escale = e[round(nrows/2),-1]
else:
    escale = e[-1,round(ncols/2)]

rel_e = e/escale

In [ ]:
vmax = np.percentile(rel_e[~np.isnan(rel_e)], 908)
fig, ax = plt.subplots()
ax.set_aspect('equal')
cmap = plt.cm.get_cmap("jet")
cmap.set_bad(color='gray')
pc = ax.pcolormesh(wt_case2.x,wt_case2.y,rel_e,shading='auto',cmap=cmap, vmax=vmax)
fig.colorbar(pc)
ax.set_xlim((0,dx*(ncols-1)))
ax.set_ylim((0,dy*(nrows-1)))
pc.set_clim((0,np.max(e[~np.isnan(e)]/escale)))
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title('Relative wave energy flux density')
fig.savefig(fig_dir / "relative_wave_energy_flux_density_1_year.png", dpi=300, bbox_inches='tight')
plt.show()

### 50 Year Retrurn Period 

In [ ]:
# define parameters for 50 year return period
period_BM = 18.32
period_POT = 17.99
mean_period = (period_BM + period_POT) / 2
period_50_year= mean_period # upper end of the POT and BM quantiles (90th percentile)
mean_wave_direction = ((259.9 + 260.4)/2) - 180 # wave direction (deg)
ang_50_year   = mean_wave_direction
nr_case1    = 100  # number of wave rays for Case 1(less rays so its easier to see the main direction of the rays)
nr_case2    = 350  # number of wave rays for Case 2(more rays so the energy flux resolution is better)
simtime = 3600*2.5 # propagate 2.5 hour
nt    = 15000 # number of time steps, try to increase to limit edge cases
print(f"successuflly initialized the wave period {period_50_year:.2f} s and wave direction {ang_50_year:.2f} deg")

In [ ]:
# ray tracing for Case 1 (less rays so its easier to see the main direction of the rays)
ang = 90-ang_50_year # convert
wt_case1 = Wave_tracing(U=np.zeros((nrows,ncols)),
                  V=np.zeros((nrows,ncols)),
                  nx=ncols, ny=nrows, nt=nt, T=simtime,
                  dx=dx,dy=dy,nb_wave_rays=nr_case1,
                  domain_X0=0, domain_XN=dx*(ncols-1),
                  domain_Y0=0, domain_YN=dy*(nrows-1),
                  d=bathy)
if np.mod(ang,360)<45 or np.mod(ang,360)>=315:
    iws = 'left'
elif np.mod(ang,360)<135 and np.mod(ang,360)>=45:
    iws = 'bottom'
elif np.mod(ang,360)<225 and np.mod(ang,360)>=135:
    iws = 'right'
else:
    iws = 'top'
wt_case1.set_initial_condition(wave_period=period_50_year,
                         theta0=+np.pi/180*ang,
                         incoming_wave_side=iws)
wt_case1.solve()

In [ ]:
wt_case2 = Wave_tracing(U=np.zeros((nrows,ncols)),
                  V=np.zeros((nrows,ncols)),
                  nx=ncols, ny=nrows, nt=nt, T=simtime,
                  dx=dx,dy=dy,nb_wave_rays=nr_case2,
                  domain_X0=0, domain_XN=dx*(ncols-1),
                  domain_Y0=0, domain_YN=dy*(nrows-1),
                  d=bathy)
if np.mod(ang,360)<45 or np.mod(ang,360)>=315:
    iws = 'left'
elif np.mod(ang,360)<135 and np.mod(ang,360)>=45:
    iws = 'bottom'
elif np.mod(ang,360)<225 and np.mod(ang,360)>=135:
    iws = 'right'
else:
    iws = 'top'
wt_case2.set_initial_condition(wave_period=period_50_year,
                         theta0=+np.pi/180*ang,
                         incoming_wave_side=iws)
wt_case2.solve()

In [ ]:
# plotting wave rays --> Case 1
fig, ax = plt.subplots()
ax.set_aspect('equal')
cmap = cmocean.cm.deep.reversed()
cmap.set_bad(color='gray')
pc=ax.pcolormesh(wt_case1.x,wt_case1.y,-wt_case1.d,shading='auto',cmap=cmap)
fig.colorbar(pc)
for ray_id in range(wt_case1.nb_wave_rays):
    ax.plot(wt_case1.ray_x[ray_id,:],wt_case1.ray_y[ray_id,:],'-k')
ax.set_xlim((0,dx*(ncols-1)))
ax.set_ylim((0,dy*(nrows-1)))
pc.set_clim((-np.max(wt_case1.d),0))
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title('Wave rays')
fig.savefig(fig_dir / "wave_rays_50_year.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# wave energy flux density estimation --> Case 2
from scipy.interpolate import RegularGridInterpolator
xx,yy,hm=wt_case1.ray_density(x_increment=20,y_increment=20)
interp = RegularGridInterpolator(points=(xx[0,:],yy[:,0]),values=hm.transpose(),bounds_error=False,fill_value=None)
xg,yg=np.meshgrid(wt_case1.x,wt_case1.y,indexing='ij')
e = interp((xg,yg))
e = e.transpose()
e[np.isnan(wt_case1.d)]=np.nan
if np.mod(ang,360)<45 or np.mod(ang,360)>=315:
    escale = e[round(nrows/2),0]
elif np.mod(ang,360)<135 and np.mod(ang,360)>=45:
    escale = e[0,round(ncols/2)]
elif np.mod(ang,360)<225 and np.mod(ang,360)>=135:
    escale = e[round(nrows/2),-1]
else:
    escale = e[-1,round(ncols/2)]

rel_e = e/escale

In [ ]:
vmax = np.percentile(rel_e[~np.isnan(rel_e)], 98)
fig, ax = plt.subplots()
ax.set_aspect('equal')
cmap = plt.cm.get_cmap("jet")
cmap.set_bad(color='gray')
pc = ax.pcolormesh(wt_case2.x,wt_case2.y,rel_e,shading='auto',cmap=cmap, vmax=vmax)
fig.colorbar(pc)
ax.set_xlim((0,dx*(ncols-1)))
ax.set_ylim((0,dy*(nrows-1)))
pc.set_clim(vmax=vmax)
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title('Relative wave energy flux density')
fig.savefig(fig_dir / "relative_wave_energy_flux_density_50_year.png", dpi=300, bbox_inches='tight')
plt.show()

## 3. Model Configuration Recap
List the main numerical settings so the reader understands what was changed relative to the base template.
- _Number of rays:_
- _Time step and total propagation time:_
- _Bathymetry/topography source:_
- _Other solver parameters you varied:_
Conclude with a sentence on whether these settings seem sufficient/realistic.

## 4. Wave Energy Flux Interpretation
Reflect on the energy density flux plots.
- _Where do rays/energy concentrate?_
- _Where do they diverge and produce smaller waves?_
- _Compare the 1-year and 50-year cases: do hotspots shift or intensify?_

## 5. Wave Height Estimates at Points of Interest
Pick at least one point (e.g., high energy region) and estimate the wave height using your model output or modified code.
- _Location (coordinates or description):_
- _Depth at that point:_
- _Estimated wave height:_
- _Applicable period:_
- _Comments on the validity of linear wave theory (cite depth-to-wavelength ratio, steepness, etc.)._

## 6. Summary Table for Floating Body Prep
Fill in the table with the case and location(s) you intend to use moving forward.
| Case | Location / Notes | Water Depth (m) | Wave Height (m) | Wave Period (s) | Wave Direction (deg / cardinal) |
| --- | --- | --- | --- | --- | --- |
| 1-year | _e.g., near offshore buoy or control point_ |  |  |  |  |
| 50-year | _e.g., same location for comparison_ |  |  |  |  |

## 7. Bonus Cross-Check (Optional)
If you had time to compare against another dataset (e.g., ResourceCode station), describe:
- _Station/data source:_
- _Observed vs. modeled differences:_
- _Potential reasons for agreement/mismatch._